<a href="https://colab.research.google.com/github/raksha-jain/LLM-Powered-Semantic-Search-Personalization-System-for-Internal-Knowledge-Base/blob/main/LLM_Powered_Semantic_Search_%26_Personalization_System_for_Internal_Knowledge_Base.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving collection (1).tar.gz to collection (1).tar.gz


In [1]:
from google.colab import files
uploaded = files.upload()

Saving qrels.train.tsv to qrels.train.tsv


In [2]:
from google.colab import files
uploaded = files.upload()

Saving queries.tar.gz to queries.tar.gz


In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter

# Load Data
passages = pd.read_csv('data/raw/collection.tsv', sep='\t', names=['pid', 'passage'], dtype={'pid': str})
queries = pd.read_csv('data/raw/queries.train.tsv', sep='\t', names=['qid', 'query'], dtype={'qid': str})
qrels = pd.read_csv('data/raw/qrels.train.tsv', sep=' ', names=['qid', '0', 'pid', 'label'], dtype={'qid': str, 'pid': str})


FileNotFoundError: [Errno 2] No such file or directory: 'data/raw/collection.tsv'